In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import itertools
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
import itertools
import math
import seaborn as sns
import scipy as sp
import pickle

In [ ]:
df_exp=pd.read_csv('../../../Downloads/CCLE_expression_22Q2.csv', index_col=0)
df_exp.columns=[i.split(' (')[0] for i in df_exp.columns]
df_tnbc_info=pd.read_csv('updated_TNBC_list_12032022.csv')
whole_genome=df_exp.columns.tolist()
df_exp_tnbc=df_exp[df_exp.index.isin(df_tnbc_info.DepMap_ID)]

In [ ]:
#To play around with the 2-dimensional datasheet with random genes
df_tmp = df_exp[['MYC', 'WNT6']]

df_tmp_t = StandardScaler().fit_transform(df_tmp)

db = DBSCAN(eps=0.2, min_samples=10).fit(df_tmp_t)
labels = db.labels_

realClusterNum = len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels))

set(labels)

labels = db.labels_

no_clusters = len(np.unique(labels) )
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % clusterNum)
print('Estimated no. of noise points: %d' % no_noise)

X = df_tmp_t

# Generate scatter plot for training data
plt.scatter(df_tmp['MYC'], df_tmp['WNT6'], c=db.labels_.astype(float), marker="o", picker=True)
plt.title('WNT6 vs MYC')
plt.xlabel('MYC')
plt.ylabel('WNT6')
plt.show()

###  The codes to run DBscan on a series of eps: 0.1-1.2, every 0.01. Together, there are 111 eps tested.

In [ ]:
list_eps=np.arange(0.1, 1.21, 0.01).tolist()
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

In [ ]:
gene_index_list=[]
eps_list=[]
cluster_number_list=[]
pan_cancer_distribution_list=[]
TNBC_distribution_list=[]

cluster_combination_list=[]
TNBC_distribution_in_this_cluster_list=[]
centroid_distance_list=[]
cluster_separation_list=[]

#2000 genes were ran for each time. For example, 1:2000, 2000:4000, 4000:6000...
for i in whole_genome[18000:]:
    for j in list_eps:
        df_tmp = df_exp[['MYC', i]]
        df_tmp_t = StandardScaler().fit_transform(df_tmp)
        db = DBSCAN(eps=j, min_samples=30).fit(df_tmp_t)
        labels = db.labels_
        df_tmp['labels']= labels
        realClusterNum = len(set(labels)) - (1 if -1 in labels else 0)
        clusterNum = len(set(labels))
        
    
    

        
        pan_cancer_distibution={}
        df_dis=df_tmp['labels'].value_counts().to_frame()
        
        for h in df_dis.index.tolist():
            pan_cancer_distibution[str(h)]=df_dis.loc[h, 'labels']
        
        
        #s_score_dict={}
        #for subset in itertools.combinations(set(labels), 2):
        #    df_test = df_tmp[(df_tmp.labels==subset[0]) | (df_tmp.labels==subset[1])]
        #    V = df_test[['MYC',i]].values
        #    v_labels = df_test['labels'].tolist()
        #    score = silhouette_score(V, v_labels, metric='euclidean')
        #    s_score_dict[subset]=score
        #silhouette_score_list.append(s_score_dict)
        
        TNBC_distibution={}
        df_tnbc_dis=df_tmp[df_tmp.index.isin(df_tnbc_info.DepMap_ID)]['labels'].value_counts().to_frame()
        for k in df_tnbc_dis.index.tolist():
            TNBC_distibution[str(k)]=df_tnbc_dis.loc[k, 'labels']

        if len(set(labels))<2:
            gene_index_list.append(i)
            eps_list.append(j)
            cluster_number_list.append(clusterNum)
            pan_cancer_distribution_list.append(pan_cancer_distibution)
            TNBC_distribution_list.append(TNBC_distibution)
            cluster_combination_list.append(np.nan)
            TNBC_distribution_in_this_cluster_list.append(np.nan)
            cluster_separation_list.append(np.nan)
            centroid_distance_list.append(np.nan)
        else:
            for subset in itertools.combinations(set(labels), 2):
                gene_index_list.append(i)
                eps_list.append(j)
                cluster_number_list.append(clusterNum)
                pan_cancer_distribution_list.append(pan_cancer_distibution)
                TNBC_distribution_list.append(TNBC_distibution)
            
                cluster_combination_list.append(subset)
            
                df_tmp_1=df_tmp[df_tmp.labels==subset[0]]
                df_tmp_2=df_tmp[df_tmp.labels==subset[1]]
            
                TNBC_in_this_cluster_distribution={}
            
                TNBC_in_this_cluster_distribution[subset[0]]=df_tmp_1[df_tmp_1.index.isin(tnbc_list)].shape[0]
                TNBC_in_this_cluster_distribution[subset[1]]=df_tmp_2[df_tmp_2.index.isin(tnbc_list)].shape[0]
            
                TNBC_distribution_in_this_cluster_list.append(TNBC_in_this_cluster_distribution)
            
                if max(df_tmp_1[i.split('_')[0]]) > max(df_tmp_2[i.split('_')[0]]):
                    cluster_separation_list.append(min(df_tmp_1[i.split('_')[0]])-max(df_tmp_2[i.split('_')[0]]))
                else:
                    cluster_separation_list.append(min(df_tmp_2[i.split('_')[0]])-max(df_tmp_1[i.split('_')[0]])) 

            
        
                centroid_1=centeroidnp(np.asanyarray(df_tmp_1[['MYC', i.split('_')[0]]]))
                centroid_2=centeroidnp(np.asanyarray(df_tmp_2[['MYC', i.split('_')[0]]]))
        
                subset_distance=math.dist(centroid_1, centroid_2)
                centroid_distance_list.append(subset_distance)
        
        
#df_results_1=pd.DataFrame(list(zip(gene_index_list, eps_list, cluster_number_list, pan_cancer_distribution_list, TNBC_distribution_list, cluster_combination_list,
#                     TNBC_distribution_in_this_cluster_list, centroid_distance_list, cluster_separation_list)),
#               columns =['gene', 'eps', 'pan_cancer_cluster_number', 'pan_cancer_distribution', 'TNBC_distribution', 'cluster_combination', 'TNBC_distribution_in_this_cluster', 'centroid_distance', 'cluster_separation'])
#df_results_1.to_csv('18000_19221_CCLE_TNBC_genome_wide_searching_for_genes_separating_cellular_states_another_method_eps_from_0.1_to_1.2_every_0.01_12172022.csv')

### The DBscan results are stored in the dataframe `df_all`.

In [ ]:
#The results for all the 19221 genes are compiled in this datasheet:
df_all=pd.read_csv('/Volumes/SSD_Yue/TNBC_MYC_heterogeneity_data/CCLE_TNBC_raw_gene_filtering_result_for_eps_0.1_to_1.2_every_0.01_across_the_whole_genome_using_another_method_12182022.csv', index_col=0)

In [ ]:
# You should drop all the columns named 'Unnamed:'.

In [ ]:
df_all.head()

In [ ]:
df_all.shape

In [ ]:
df_all['pan_cancer_distribution']=df_all['pan_cancer_distribution'].apply(lambda x: eval(x))

In [ ]:
df_all['TNBC_distribution']=df_all['TNBC_distribution'].apply(lambda x: eval(x))

In [ ]:
df_all=df_all[~df_all.TNBC_distribution_in_this_cluster.isnull()]

In [ ]:
df_all['TNBC_distribution_in_this_cluster']=df_all['TNBC_distribution_in_this_cluster'].apply(lambda x: eval(x))

In [ ]:
df_all['cluster_combination']=df_all['cluster_combination'].apply(lambda x: eval(x))

### Filter 1: To filter out genes that have max expressions among TNBC cell lines that are lower than 3; To filter out genes with expression differences among TNBC cell lines that are lower than 3

In [ ]:
list_1=[]
for i in whole_genome:
    if df_exp_tnbc[i].max() >3:
        if df_exp_tnbc[i].max()- df_exp_tnbc[i].min() >3:
            list_1.append(i)

In [ ]:
len(list_1)

In [ ]:
df_selection_1=df_all[df_all.gene.isin(list_1)]
df_selection_1.reset_index(drop=True, inplace=True)

In [ ]:
df_selection_1.shape

In [ ]:
df_selection_1.head()

### Filter 2: To select genes with at least one cluster combination where at least 3 TNBC cell lines in each cluster, and centroid distance above 3 and cluster separation above 1.

In [ ]:
df_selection_2= df_selection_1[~df_selection_1['TNBC_distribution_in_this_cluster'].apply(lambda x: -1 in x.keys())]

In [ ]:
df_selection_2.head()

In [ ]:
df_selection_2.shape

In [ ]:
df_selection_3=df_selection_2[df_selection_2['TNBC_distribution_in_this_cluster'].apply(lambda x: [x[key]>=3 for key  in list(x.keys())]== [True, True])]

In [ ]:
df_selection_4=df_selection_3[(df_selection_3.centroid_distance_for_this_cluster_combination>3) & (df_selection_3.cluster_separation_for_this_cluster_combination>1)]

In [ ]:
df_selection_4.shape

In [ ]:
df_selection_4.head()

In [ ]:
len(df_selection_4.gene.unique().tolist())

In [ ]:
list_gene_selected= df_selection_4.gene.unique().tolist()

In [ ]:
df_selection_4.to_csv('CCLE_TNBC_cell_lines_cellular_state_separating_gene_selection_results_12202022.csv')

In [ ]:
#To visualize the selected genes:
df1=pd.read_csv('CCLE_TNBC_cell_lines_cellular_state_separating_gene_selection_results_12202022.csv')
df1=df1.drop_duplicates('gene', keep='first')
df1['combination']=df1.apply(lambda x: str(x['gene']) + '_' + str(x['eps']), axis=1)
list_to_plot=df1.combination.tolist()
for i in list_to_plot:
    df_tmp = df_exp[['MYC', i.split('_')[0]]]
    df_tmp_t = StandardScaler().fit_transform(df_tmp)
    db = DBSCAN(eps=float(i.split('_')[1]), min_samples=30).fit(df_tmp_t)
    labels = db.labels_
    df_tmp['labels']= labels
    df_tmp['alpha']=0.1
    df_tmp['color']=df_tmp['labels'].map(dict_map)
    list_labels=[i for i in df_tmp.labels.unique().tolist() if i is not -1]
    
    for j in df_tmp.index.tolist():
        if j in list_except:
            df_tmp.loc[j, 'color']='blue'
            df_tmp.loc[j, 'alpha']=1
        elif j in list_scRNA_available:
            df_tmp.loc[j, 'color']='red'
            df_tmp.loc[j, 'alpha']=1
    
    fig, ax = plt.subplots()
    ax.scatter(df_exp['MYC'], df_exp[i.split('_')[0]], c=df_tmp['color'].tolist(), alpha=df_tmp['alpha'].tolist())
    plt.xlabel("MYC")
    plt.ylabel(i.split('_')[0])

    for k, txt in enumerate(list_annotation):
        ax.annotate(txt, (df_exp['MYC'][k], df_exp[i.split('_')[0]][k]), fontsize=7)
   # for h in list_labels:
    #    ax.annotate('cluster {} : '.format(h) + '{}'.format(df_TNBC_cell_line_cluster_assignment.loc[df_TNBC_cell_line_cluster_assignment.gene==i.split('_')[0], 'cluster_{}'.format(h)].tolist()[0]), (0, df_exp[i.split('_')[0]].max()-(h+0.15)), fontsize=7)
   # plt.savefig('./399_filtered_genes_after_using_a_denser_eps_series_across_the_whole_genome_for_updated_30_TNBC_cell_lines_12032022/{}_vs_MYC_gene_expression_12032022.png'.format(i.split('_')[0]), dpi=150)